# 15 LoopStack

In [1]:
# default_exp loopstack

In [1]:
# export
from forgebox.loop import Loop,ProgressBar,Tolerate,Event
from types import MethodType

In [2]:
# export
def create_event(event_name):
    class BatchEvent(Event):pass
    BatchEvent.__name__ = event_name
    return BatchEvent

def events(*enames):
    return list(map(create_event,enames))

In [3]:
# export
class LoopStack(Loop):
    @classmethod
    def from_loops(cls,*loops):
        def init(self_,iterable=[],name = None):
            name = name if name!=None else cls.__name__
            super().__init__(iterable = iterable,
                             name = name)
            self_.loops = dict()
            l = None
            for L in loops:
          #                 setattr(self,L.__name__,l)      
                if l==None:
                    l = L(iterable = self_)
                else:
                    l = L(iterable = l)
                print(l.core.forall_pool.keys())
                self_.loops.update({L.__name__:l})
            
        setattr(cls,"__init__",init)
        return cls
    
    def __repr__(self,):
        return f"LoopStack>:{self.name}\n\t"+\
            "\n\t".join(map(lambda x:"Layer>>:"+x,self.loops.keys()))

In [4]:
TRAIN_EVENTS = ["DATA_PROCESS","FORWARD","LOSS_CALC","BACKWARD","OPT_STEP","METRICS"]
EVAL_EVENTS = ["DATA_PROCESS","EVAL_WRAP","FORWARD","LOSS_CALC","METRICS"]

In [5]:
class TrainLoop(LoopStack):
    def __init__(self,data_iter):
        self.from_loops(ProgressBar,Tolerate,
                        *events(*TRAIN_EVENTS))
        self.__init__(data_iter,)

In [6]:
loop = TrainLoop(range(20))
loop

dict_keys(['pgbar_data', 'pgbar_description'])
dict_keys(['pgbar_data', 'pgbar_description', 'error_list'])
dict_keys(['pgbar_data', 'pgbar_description', 'error_list'])
dict_keys(['pgbar_data', 'pgbar_description', 'error_list'])
dict_keys(['pgbar_data', 'pgbar_description', 'error_list'])
dict_keys(['pgbar_data', 'pgbar_description', 'error_list'])
dict_keys(['pgbar_data', 'pgbar_description', 'error_list'])
dict_keys(['pgbar_data', 'pgbar_description', 'error_list'])


LoopStack>:TrainLoop
	Layer>>:ProgressBar
	Layer>>:Tolerate
	Layer>>:DATA_PROCESS
	Layer>>:FORWARD
	Layer>>:LOSS_CALC
	Layer>>:BACKWARD
	Layer>>:OPT_STEP
	Layer>>:METRICS

In [7]:
id(loop.core)

4618034896

In [121]:
loop.core.assign_forall(loop)

In [125]:
loop.core.update_forall(loop)
loop.core.forall_pool

{'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>,
 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>,
 'error_list': <bound method Tolerate.error_list of layer:>>>Tolerate>}

In [130]:
loop = loop.next_layer
print(loop.core.for_all_functions(loop))
loop.core.update_forall(loop)
loop.core.forall_pool

{'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>, 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>}


{'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>,
 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>,
 'error_list': <bound method Tolerate.error_list of layer:>>>Tolerate>}

In [131]:
loop = loop.next_layer
print(loop.core.for_all_functions(loop))
loop.core.update_forall(loop)
loop.core.forall_pool

{'error_list': <bound method Tolerate.error_list of layer:>>>Tolerate>, 'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>, 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>}


{'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>,
 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>,
 'error_list': <bound method Tolerate.error_list of layer:>>>Tolerate>}

In [132]:
loop = loop.next_layer
print(loop.core.for_all_functions(loop))
loop.core.update_forall(loop)
loop.core.forall_pool

{'after_last_DATA_PROCESS': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>DATA_PROCESS>, 'before_1st_DATA_PROCESS': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>DATA_PROCESS>, 'error_list': <bound method Tolerate.error_list of layer:>>>Tolerate>, 'on_DATA_PROCESS': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>DATA_PROCESS>, 'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>, 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>}


{'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>,
 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>,
 'error_list': <bound method Tolerate.error_list of layer:>>>Tolerate>,
 'after_last_DATA_PROCESS': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>DATA_PROCESS>,
 'before_1st_DATA_PROCESS': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>DATA_PROCESS>,
 'on_DATA_PROCESS': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>DATA_PROCESS>}

In [133]:
loop = loop.next_layer
print(loop.core.for_all_functions(loop))
loop.core.update_forall(loop)
loop.core.forall_pool

{'after_last_FORWARD': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>FORWARD>, 'before_1st_FORWARD': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>FORWARD>, 'error_list': <bound method Tolerate.error_list of layer:>>>Tolerate>, 'on_FORWARD': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>FORWARD>, 'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>, 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>}


{'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>,
 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>,
 'error_list': <bound method Tolerate.error_list of layer:>>>Tolerate>,
 'after_last_DATA_PROCESS': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>DATA_PROCESS>,
 'before_1st_DATA_PROCESS': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>DATA_PROCESS>,
 'on_DATA_PROCESS': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>DATA_PROCESS>,
 'after_last_FORWARD': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>FORWARD>,
 'before_1st_FORWARD': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>FORWARD>,
 'on_FORWARD': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>FORWARD>}

In [134]:
loop = loop.next_layer
print(loop.core.for_all_functions(loop))
loop.core.update_forall(loop)
loop.core.forall_pool

{'after_last_LOSS_CALC': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>LOSS_CALC>, 'before_1st_LOSS_CALC': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>LOSS_CALC>, 'error_list': <bound method Tolerate.error_list of layer:>>>Tolerate>, 'on_LOSS_CALC': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>LOSS_CALC>, 'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>, 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>}


{'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>,
 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>,
 'error_list': <bound method Tolerate.error_list of layer:>>>Tolerate>,
 'after_last_DATA_PROCESS': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>DATA_PROCESS>,
 'before_1st_DATA_PROCESS': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>DATA_PROCESS>,
 'on_DATA_PROCESS': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>DATA_PROCESS>,
 'after_last_FORWARD': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>FORWARD>,
 'before_1st_FORWARD': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>FORWARD>,
 'on_FORWARD': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>FORWARD>,
 'after_last_LOSS_CALC': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>LOSS_CALC>,
 'before_1st_LOSS_CALC': <bound method Event.create_cb_deco.<locals>

In [122]:
loop.next_layer.next_layer.next_layer.core.forall_pool

{'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>,
 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>,
 'error_list': <bound method Tolerate.error_list of layer:>>>Tolerate>}

In [123]:
loop.core.for_all_functions(loop.next_layer.next_layer.next_layer.next_layer.next_layer)

{'after_last_LOSS_CALC': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>LOSS_CALC>,
 'before_1st_LOSS_CALC': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>LOSS_CALC>,
 'error_list': <bound method Tolerate.error_list of layer:>>>Tolerate>,
 'on_LOSS_CALC': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>LOSS_CALC>,
 'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>,
 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>}

In [124]:
@loop.on_PROCESS
def process_data(loop):
    loop.core.double = loop.element*2
    
@loop.on_FORWARD
def update_pg(loop):
    loop.pgbar_description(f"double:{self.double}")

AttributeError: 'StorageCore' object has no attribute 'on_PROCESS'

In [59]:
loop.core.for_all_functions(loop.OPT_STEP)

{'after_last_OPT_STEP': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>OPT_STEP>,
 'before_1st_OPT_STEP': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>OPT_STEP>,
 'error_list': <bound method Tolerate.error_list of layer:>>>Tolerate>,
 'on_OPT_STEP': <bound method Event.create_cb_deco.<locals>.wraper of layer:>>>OPT_STEP>,
 'pgbar_data': <bound method ProgressBar.pgbar_data of layer:>>>ProgressBar>,
 'pgbar_description': <bound method ProgressBar.pgbar_description of layer:>>>ProgressBar>}

In [53]:
loop.summary()

Loop layer TrainLoop summary:
🍰layer0.0	LoopStack>:TrainLoop
	Layer>>:ProgressBar
	Layer>>:Tolerate
	Layer>>:PROCESS
	Layer>>:FORWARD
	Layer>>:LOSS_CALC
	Layer>>:BACKWARD
	Layer>>:OPT_STEP
	Layer>>:METRICS
	[🐍func_name]	after_last_METRICS
	[⛰doc]	
            Append new after_last callback for event:METRICS
            Use this function as decorator
        
	[😝var]	cls,f
	[😜names]	getattr
	...................................
	[🐍func_name]	before_1st_METRICS
	[⛰doc]	
            Append new before_1st callback for event:METRICS
            Use this function as decorator
        
	[😝var]	cls,f
	[😜names]	getattr
	...................................
	[🐍func_name]	error_list
	[⛰doc]	
        A list of errors happend so far
        
	[😝var]	self
	[😜names]	errors
	...................................
	[🐍func_name]	on_METRICS
	[⛰doc]	
            Append new on callback for event:METRICS
            Use this function as decorator
        
	[😝var]	cls,f
	[😜names]	getattr
	........................

In [11]:
train_loop.run()

In [54]:
loopstack = LoopStack\
    .from_loops(*events("a","b","c"))(range(5))

In [55]:
loopstack

LoopStack>:LoopStack
	Layer>>:a
	Layer>>:b
	Layer>>:c